### Note collection_day refers to day of most recent attempt at collecting certificate, but such attempt may not have yielded a certificate anyways. Actual count of certificates is done elsewhere.

In [1]:
from OpenSSL import crypto
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import os
import shutil

In [2]:
base_path = "url_datasets/phishtank-"
days = ['16-04','18-04','19-04','20-04','21-04','22-04','23-04','24-04','30-04','01-05','03-05','04-05','05-05','07-05','08-05']

In [3]:
dfs = []
for day in days:
    path = base_path + day + '.csv'
    df = pd.read_csv(path)
    dfs.append(df)

In [4]:
distinct_urls = []
distinct_url_dfs = []
for i, df in enumerate(dfs):
    if i > 0:
        my_urls = df['url']
        prev_urls = dfs[i-1]['url']
        new_urls = list(set(my_urls)-set(prev_urls))
    else:
        new_urls = df['url']
    print(len(new_urls))
    new_df = pd.DataFrame()
    new_df['url'] = new_urls
    new_df['day_collected'] = [days[i] for _ in new_urls]
    distinct_url_dfs.append(new_df)
    
    distinct_urls.append(new_urls)

10170
706
426
361
382
423
734
55
1453
437
738
465
741
727


In [5]:
final_distinct_df = pd.concat(distinct_url_dfs)
final_distinct_df

,url,day_collected
0,https://microsoftonlinefaxlinkvonage.weebly.com/,16-04
1,http://bunblkol.com/,16-04
2,https://acibadem.weebly.com/,16-04
3,http://ingdericto.temp.swtest.ru/ing/,16-04
4,https://geolp.weebly.com/,16-04
...,...,...
722,https://amaeoirugleriwserwermbvfdzv.life/pc,07-05
723,https://economy.freedynamicdns.net/,07-05
724,http://p7.continuewide.work/apmix,07-05
725,http://domamaxion.com/DONE/amazkn/mazon/3db5c/...,07-05


In [6]:
final_distinct_df.to_csv('url_datasets/distinct_phish_urls.csv')

In [7]:
len(set(final_distinct_df['url']))

17817

# reorganise all phish data in one folder

In [8]:
#cert collection might have succeded on previous day and failed on next, there were mistakes early on where certificates were recollected. Newest certificate for a url will be kept.

In [9]:
for i,d in enumerate(days):
    data_path = 'phish_data-'+d+'/'
    target_path = 'phish_data/'
    curr_urls = dfs[i]['url']
    
    for url in curr_urls:
        try:
            url = url.split("/")[2]
            f = open(data_path+url)
            line = f.readline()
            if line != '':
                f.close()
                shutil.move(data_path+url, target_path+url)
        except:
            pass
        

# sort out benign collection tracker

In [23]:
# even benign collection fails sometimes, so the 11000 failures were reattempted on another day, need to track which day they were collected though for certain statistics

In [2]:
benign_days = ['04-05','07-05']

In [3]:
path = "url_datasets/majestic_million.csv"
df = pd.read_csv(path)
urls = df['Domain']

In [4]:
how_many = 50000
urls = urls[:50000]

In [16]:
failures_i = np.zeros(how_many,dtype=bool)
failures = []
f = open('benign_urls-refetch.txt')
line = f.readline()[:-1]
while line != '':
    failures.append(line)
    line = f.readline()[:-1]
f.close()



for i in range(how_many):
    if urls[i] in failures:
        failures_i[i] = True

In [17]:
Counter(failures_i)

Counter({False: 38010, True: 11990})

In [20]:
day_tags = [benign_days[1] if failures_i[i] else benign_days[0] for i in range(how_many)]

benign_df = pd.DataFrame()
benign_df['url'] = urls
benign_df['day_collected'] = day_tags

In [21]:
benign_df

,url,day_collected
0,google.com,04-05
1,facebook.com,07-05
2,youtube.com,04-05
3,twitter.com,04-05
4,instagram.com,07-05
...,...,...
49995,ssoloans.com,07-05
49996,sildenafilyes.com,04-05
49997,pillmy.com,04-05
49998,bradshawfoundation.com,07-05


In [24]:
benign_df.to_csv('url_datasets/distinct_benign_urls.csv')